In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#input_file = "output_20191008_0336AM.csv" #complete
#input_file = "output_20191116_1256PM.csv" #partial
input_file = "output_20200219_2125PM.csv" #double
df = pd.read_csv(input_file, header = 0)

In [3]:
df['FLOW'] = df['SRCNODE']+'-'+df['DESTNODE']
df['FM']=df['FAILURE']+df['MISSING']

In [4]:
df_failure = df[ (df.FM==1)]

In [5]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier
from sklearn.calibration import CalibratedClassifierCV

In [6]:
df_dummy = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST'], columns=['SRCNODE', 'DESTNODE'])

In [7]:
y=df_failure['LABEL']
x=df_dummy.drop(['LABEL'],axis=1)

In [8]:
linear_svc=svm.LinearSVC(random_state=0)
lin_clf=linear_svc.fit(x, y)

In [9]:
cal_svc=CalibratedClassifierCV(base_estimator=linear_svc,cv="prefit")
cal_clf=cal_svc.fit(x,y)

In [10]:
%%time
ovo_lin_model=OneVsOneClassifier(svm.LinearSVC(random_state=0)).fit(x, y)

CPU times: user 11.6 s, sys: 50.6 ms, total: 11.6 s
Wall time: 11.2 s


In [11]:
%%time
#svm_pred=svm_model.predict(x)
lin_df = ovo_lin_model.decision_function(x)

CPU times: user 48.7 s, sys: 1.25 s, total: 50 s
Wall time: 17.8 s


In [12]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.LABEL==label]
        if test_data_1.shape[0]==0:
            print("No match")
            continue
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            correct=correct+1
    return correct, correct/num

In [13]:
c,r=accuracy(cal_clf, df_dummy, ovo_lin_model.classes_,1)
c,r

(40, 0.6060606060606061)

In [14]:
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score
y_pred_rf = cal_clf.predict(x)
print(balanced_accuracy_score(y, y_pred_rf))
f1_score(y, y_pred_rf,average='weighted')

0.2463278028318829


/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3680773409448971

In [15]:
c,r=accuracy(lin_df, df_dummy, ovo_lin_model.classes_,10)
c,r

AttributeError: 'numpy.ndarray' object has no attribute 'predict_proba'

In [16]:
ovo_svm_model=OneVsOneClassifier(svm.SVC(random_state=0)).fit(x, y)
#svm_pred=svm_model.predict(x)
svm_df = ovo_svm_model.decision_function(x)

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

...

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/yxin/anaconda3/lib/python3.7/site-packages/sklearn/

In [17]:
def accuracy_ori(model, test_data, classes):
    num = classes.size
    correct=0
    for label in classes:
        test_data_1=test_data[test_data.LABEL==label]
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.decision_function(x_test_1)
        label_index=test_prob_1.sum(axis=0).argmax()
        label_pred = classes[label_index]
        if(label==label_pred):
            correct=correct+1
    return correct, correct/num

In [18]:
c,r=accuracy_ori(lin_df, df_dummy, ovo_lin_model.classes_)
c,r

AttributeError: 'numpy.ndarray' object has no attribute 'decision_function'

In [19]:
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [20]:
multilabel_confusion_matrix(y, svm_pred)

NameError: name 'svm_pred' is not defined

In [ ]:
cross_val_score(clf_svm, x, y, scoring='accuracy',cv=5)

In [ ]:
cross_val_score(clf_linsvm, x, y, scoring='accuracy',cv=5)

In [ ]:
f1_score(y, lin_pred,average='weighted')

In [ ]:
f1_score(y, svm_pred,average='weighted')

In [ ]:
clf_svm.score(x,y)

In [ ]:
clf_linsvm.score(x,y)

In [ ]:
balanced_accuracy_score(y, svm_pred)

In [ ]:
balanced_accuracy_score(y, lin_pred)

In [ ]:
x.values.shape

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

In [ ]:
classifier = BinaryRelevance(classifier = SVC(probability=True),
    require_dense = [False, True])

classifier.fit(x.to_numpy(), y.to_numpy())

In [21]:
test_file = "output_20191008_0336AM.csv" #complete
#test_file = "big.csv" #partial
test_df = pd.read_csv(test_file, header = 0)
test_df['FLOW'] = test_df['SRCNODE']+'-'+test_df['DESTNODE']
test_df['FM']=test_df['FAILURE']+test_df['MISSING']
test_df_failure = test_df[ (test_df.FM==1)]

test_df_dummy = pd.get_dummies(test_df_failure[['SRCNODE', 'DESTNODE','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST'], columns=['SRCNODE', 'DESTNODE'])
test_y=test_df_failure['LABEL']
test_x=test_df_dummy.drop(['LABEL'],axis=1)

In [25]:
%%time
c,r=accuracy(cal_clf, test_df_dummy, cal_clf.classes_,3)
c,r

No match
No match
No match
No match
No match
CPU times: user 1.4 s, sys: 44.4 ms, total: 1.44 s
Wall time: 373 ms


(36, 0.5454545454545454)